# PDF RAG 系统 - 使用 BGE-M3 Embedding 模型

本 Notebook 展示如何使用 XPULink 平台托管的 BGE-M3 Embedding 模型构建一个基于 PDF 文档的 RAG（检索增强生成）系统。

## 功能特性
- 加载和处理 PDF 文档
- 使用 BGE-M3 Embedding 模型进行文档向量化
- 构建向量索引实现高效检索
- 基于检索结果生成智能回答

## 1. 环境配置和依赖安装

In [ ]:
# 安装必要的依赖包
!pip install llama-index-core llama-index-llms-openai python-dotenv pypdf

In [ ]:
import os
import json
import requests
from typing import List, Optional
from dotenv import load_dotenv

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.embeddings import BaseEmbedding
from llama_index.llms.openai import OpenAI

# 加载环境变量
load_dotenv()

# 检查 API Key
if os.getenv('XPU_API_KEY'):
    print("✅ 找到 XPU_API_KEY 环境变量")
else:
    raise ValueError("❌ 未找到 XPU_API_KEY。请在 .env 文件中设置后再运行此脚本。")

## 2. 定义 BGE-M3 Embedding 类

BGE-M3 是一个强大的多语言 Embedding 模型，支持中英文等多种语言，特别适合处理中文文档。

In [ ]:
class BGEM3Embedding(BaseEmbedding):
    """BGE-M3 Embedding 模型实现（基于 OpenAI 兼容 API）"""
    
    def __init__(
        self,
        api_base: str = "https://xpulink.ai/v1",
        api_key: Optional[str] = None,
        model: str = "bge-m3",
        embed_batch_size: int = 10,
        **kwargs
    ) -> None:
        """
        初始化 BGE-M3 Embedding 模型
        
        Args:
            api_base: XPULink API 基础地址
            api_key: API 密钥（从环境变量获取）
            model: 模型名称，默认为 bge-m3
            embed_batch_size: 批处理大小
        """
        self.api_base = api_base.rstrip('/')
        self.api_key = api_key or os.getenv("XPU_API_KEY")
        self.model = model
        self.embed_batch_size = embed_batch_size
        
        if not self.api_key:
            raise ValueError("需要提供 API Key")
        
        super().__init__(**kwargs)
    
    def _call_api(self, texts: List[str]) -> List[List[float]]:
        """调用 XPULink API 获取 embeddings"""
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        
        data = {
            'model': self.model,
            'input': texts
        }
        
        try:
            response = requests.post(
                f"{self.api_base}/embeddings",
                headers=headers,
                json=data,
                timeout=60
            )
            response.raise_for_status()
            
            result = response.json()
            if result.get('data'):
                return [item['embedding'] for item in result['data']]
            else:
                raise Exception(f"API 返回格式错误: {result}")
                
        except requests.exceptions.RequestException as e:
            raise Exception(f"API 请求失败: {str(e)}")
    
    def _get_query_embedding(self, query: str) -> List[float]:
        """获取单个查询的 embedding"""
        embeddings = self._call_api([query])
        return embeddings[0] if embeddings else []
    
    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        """批量获取文本的 embeddings"""
        all_embeddings = []
        
        # 分批处理
        for i in range(0, len(texts), self.embed_batch_size):
            batch = texts[i:i + self.embed_batch_size]
            batch_embeddings = self._call_api(batch)
            all_embeddings.extend(batch_embeddings)
            
            if i + self.embed_batch_size < len(texts):
                print(f"已处理 {i + len(batch)}/{len(texts)} 个文本片段")
        
        return all_embeddings
    
    async def _aget_query_embedding(self, query: str) -> List[float]:
        """异步获取查询 embedding（回退到同步方法）"""
        return self._get_query_embedding(query)
    
    @property
    def model_name(self) -> str:
        return self.model

print("✅ BGE-M3 Embedding 类定义完成")

## 3. 配置 LlamaIndex Settings

配置全局的 Embedding 模型和 LLM。

In [ ]:
# 配置 BGE-M3 Embedding 模型
Settings.embed_model = BGEM3Embedding(
    api_base="https://xpulink.ai/v1",
    model="bge-m3",
    embed_batch_size=10
)

# 配置 LLM（用于生成回答）
Settings.llm = OpenAI(
    api_key=os.getenv("XPU_API_KEY"),
    api_base="https://www.xpulink.ai/v1",
    model="qwen3-32b",
    temperature=0.7
)

print("✅ LlamaIndex 配置完成")
print(f"  - Embedding 模型: {Settings.embed_model.model_name}")
print(f"  - LLM 模型: {Settings.llm.model}")

## 4. 加载 PDF 文档

将 PDF 文件放在 `data/` 目录下，系统会自动加载并处理。

In [ ]:
# 设置数据目录路径
DATA_DIR = "./data/"

# 确保数据目录存在
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)
    print(f"📁 已创建数据目录: {DATA_DIR}")
    print("⚠️  请将 PDF 文件放入此目录后重新运行此单元格")
else:
    # 加载文档
    try:
        documents = SimpleDirectoryReader(
            input_dir=DATA_DIR,
            required_exts=[".pdf"]
        ).load_data()
        
        if documents:
            print(f"✅ 成功加载 {len(documents)} 个文档片段")
            print(f"\n📄 第一个文档片段预览:")
            print(f"  - 长度: {len(documents[0].text)} 字符")
            print(f"  - 内容预览: {documents[0].text[:200]}...")
        else:
            print(f"⚠️  未在 {DATA_DIR} 目录中找到 PDF 文件")
            print("请添加 PDF 文件后重新运行此单元格")
            
    except Exception as e:
        print(f"❌ 加载文档失败: {str(e)}")

## 4.5 预测向量数据库大小

在构建索引之前，预测向量数据库所需的存储空间，帮助您规划资源。

In [ ]:
def estimate_vector_db_size(documents, embedding_dim=1024, dtype_bytes=4):
    """
    预测向量数据库的存储大小
    
    Args:
        documents: 文档列表
        embedding_dim: 向量维度（BGE-M3 默认为 1024）
        dtype_bytes: 数据类型字节数（float32 为 4 字节）
    
    Returns:
        dict: 包含各项大小统计的字典
    """
    if not documents:
        return {
            'error': '没有文档可供分析',
            'total_size_mb': 0
        }
    
    # 估算文档被分块的数量
    # LlamaIndex 默认 chunk_size=1024, chunk_overlap=20
    chunk_size = 1024
    chunk_overlap = 20
    effective_chunk_size = chunk_size - chunk_overlap
    
    total_chars = sum(len(doc.text) for doc in documents)
    estimated_chunks = max(1, total_chars // effective_chunk_size)
    
    # 计算向量存储大小
    # 每个 chunk 需要一个 embedding 向量
    vector_size_bytes = estimated_chunks * embedding_dim * dtype_bytes
    vector_size_mb = vector_size_bytes / (1024 * 1024)
    
    # 计算文本存储大小（UTF-8 编码，约每字符 2-3 字节，这里取 2.5）
    text_size_bytes = total_chars * 2.5
    text_size_mb = text_size_bytes / (1024 * 1024)
    
    # 计算元数据存储大小（估算每个 chunk 约 500 字节元数据）
    metadata_size_bytes = estimated_chunks * 500
    metadata_size_mb = metadata_size_bytes / (1024 * 1024)
    
    # 索引开销（FAISS 或其他索引结构，约为向量大小的 20-30%）
    index_overhead_factor = 0.25
    index_overhead_mb = vector_size_mb * index_overhead_factor
    
    # 总大小
    total_size_mb = vector_size_mb + text_size_mb + metadata_size_mb + index_overhead_mb
    
    return {
        'total_documents': len(documents),
        'total_characters': total_chars,
        'estimated_chunks': estimated_chunks,
        'embedding_dimension': embedding_dim,
        'vector_storage_mb': round(vector_size_mb, 2),
        'text_storage_mb': round(text_size_mb, 2),
        'metadata_storage_mb': round(metadata_size_mb, 2),
        'index_overhead_mb': round(index_overhead_mb, 2),
        'total_size_mb': round(total_size_mb, 2),
        'total_size_gb': round(total_size_mb / 1024, 3)
    }


def print_size_estimation(estimation):
    """打印格式化的大小预测结果"""
    if 'error' in estimation:
        print(f"❌ {estimation['error']}")
        return
    
    print("=" * 60)
    print("📊 向量数据库大小预测")
    print("=" * 60)
    print(f"\n📄 文档统计:")
    print(f"  - 文档数量: {estimation['total_documents']}")
    print(f"  - 总字符数: {estimation['total_characters']:,}")
    print(f"  - 预计分块数: {estimation['estimated_chunks']:,}")
    print(f"  - 向量维度: {estimation['embedding_dimension']}")
    
    print(f"\n💾 存储空间预测:")
    print(f"  - 向量存储: {estimation['vector_storage_mb']:,.2f} MB")
    print(f"  - 文本存储: {estimation['text_storage_mb']:,.2f} MB")
    print(f"  - 元数据存储: {estimation['metadata_storage_mb']:,.2f} MB")
    print(f"  - 索引开销: {estimation['index_overhead_mb']:,.2f} MB")
    
    print(f"\n📦 总计:")
    print(f"  - 总大小: {estimation['total_size_mb']:,.2f} MB ({estimation['total_size_gb']:.3f} GB)")
    
    # 添加建议
    total_mb = estimation['total_size_mb']
    print(f"\n💡 建议:")
    if total_mb < 100:
        print("  ✅ 内存占用较小，可以轻松在内存中处理")
    elif total_mb < 1000:
        print("  ⚠️  内存占用适中，确保有足够的可用内存")
    else:
        print("  ⚠️  内存占用较大，建议考虑:")
        print("     - 使用持久化向量数据库（如 Chroma、Weaviate）")
        print("     - 分批处理文档")
        print("     - 使用更强大的服务器")
    
    print("=" * 60)


# 如果已加载文档，进行大小预测
if 'documents' in locals() and documents:
    print("🔄 正在分析文档并预测向量数据库大小...\n")
    
    estimation = estimate_vector_db_size(documents)
    print_size_estimation(estimation)
    
    # 保存预测结果供后续参考
    db_size_estimation = estimation
    
else:
    print("⚠️  请先运行上一个单元格加载文档")

## 5. 构建向量索引

使用 BGE-M3 模型对文档进行向量化，并构建索引以支持高效检索。

In [ ]:
if 'documents' in locals() and documents:
    # 显示预测信息（如果有的话）
    if 'db_size_estimation' in locals():
        print(f"📊 预计向量数据库大小: {db_size_estimation['total_size_mb']:.2f} MB")
        print(f"   预计处理 {db_size_estimation['estimated_chunks']:,} 个文本块\n")
    
    print("🔄 开始构建向量索引...")
    print("   这可能需要几分钟时间，取决于文档大小\n")
    
    try:
        # 构建向量索引
        index = VectorStoreIndex.from_documents(
            documents,
            show_progress=True
        )
        
        print("\n✅ 向量索引构建完成！")
        print("   现在可以进行文档查询了")
        
        # 如果有预测信息，可以进行比较
        if 'db_size_estimation' in locals():
            print(f"\n💾 预测的数据库大小: {db_size_estimation['total_size_mb']:.2f} MB")
            print("   （实际内存使用可能会有所不同）")
        
    except Exception as e:
        print(f"❌ 构建索引失败: {str(e)}")
else:
    print("⚠️  请先运行上一个单元格加载文档")

## 6. 创建查询引擎

创建查询引擎，支持基于文档内容的智能问答。

In [ ]:
if 'index' in locals():
    # 创建查询引擎
    query_engine = index.as_query_engine(
        similarity_top_k=3,  # 返回最相似的 3 个片段
        response_mode="compact"  # 使用紧凑模式生成回答
    )
    
    print("✅ 查询引擎创建完成")
    print("\n使用方式:")
    print("  response = query_engine.query('你的问题')")
    print("  print(response)")
else:
    print("⚠️  请先运行上一个单元格构建索引")

## 7. 示例查询

运行一些示例查询来测试 RAG 系统。

In [ ]:
# 示例查询 1
if 'query_engine' in locals():
    query = "文档的主要内容是什么？"
    print(f"🔍 查询: {query}\n")
    
    try:
        response = query_engine.query(query)
        print("💡 回答:")
        print(response)
        
        print("\n📚 相关文档片段:")
        for i, node in enumerate(response.source_nodes, 1):
            print(f"\n  片段 {i} (相似度: {node.score:.4f}):")
            print(f"  {node.text[:200]}...")
            
    except Exception as e:
        print(f"❌ 查询失败: {str(e)}")
else:
    print("⚠️  请先创建查询引擎")

In [ ]:
# 示例查询 2 - 自定义查询
if 'query_engine' in locals():
    # 修改这里的查询内容
    custom_query = "请总结文档中的关键要点"
    
    print(f"🔍 查询: {custom_query}\n")
    
    try:
        response = query_engine.query(custom_query)
        print("💡 回答:")
        print(response)
        
    except Exception as e:
        print(f"❌ 查询失败: {str(e)}")
else:
    print("⚠️  请先创建查询引擎")

## 8. 交互式查询

创建一个简单的交互式界面，方便进行多轮查询。

In [ ]:
def interactive_query():
    """交互式查询函数"""
    if 'query_engine' not in locals() and 'query_engine' not in globals():
        print("⚠️  请先创建查询引擎")
        return
    
    print("="*50)
    print("📖 PDF RAG 交互式查询系统")
    print("="*50)
    print("输入 'exit' 或 'quit' 退出\n")
    
    while True:
        try:
            query = input("\n🔍 请输入您的问题: ").strip()
            
            if query.lower() in ['exit', 'quit', '退出']:
                print("\n👋 再见！")
                break
            
            if not query:
                continue
            
            print("\n💭 思考中...\n")
            response = query_engine.query(query)
            
            print("💡 回答:")
            print(response)
            print("\n" + "-"*50)
            
        except KeyboardInterrupt:
            print("\n\n👋 再见！")
            break
        except Exception as e:
            print(f"\n❌ 查询出错: {str(e)}")

# 取消注释下面这行来启动交互式查询
# interactive_query()

## 总结

本 Notebook 展示了如何使用 XPULink 平台上的 BGE-M3 Embedding 模型构建一个完整的 PDF RAG 系统：

1. ✅ 配置了 BGE-M3 Embedding 模型
2. ✅ 加载和处理 PDF 文档
3. ✅ **预测向量数据库大小**（新增功能）
4. ✅ 构建了向量索引
5. ✅ 实现了智能查询功能
6. ✅ 提供了交互式查询界面

### 向量数据库大小预测功能

新增的预测功能可以帮助您：
- 📊 在构建索引前预估所需的存储空间
- 💾 了解文档会被分成多少个文本块
- 🎯 获取针对不同数据规模的优化建议
- ⚡ 更好地规划资源和选择合适的向量数据库

预测考虑了以下因素：
- 向量存储空间（BGE-M3 使用 1024 维向量）
- 文本内容存储
- 元数据开销
- 索引结构开销

### 下一步

- 可以尝试不同的 PDF 文档
- 调整 `similarity_top_k` 参数来控制检索的文档数量
- 修改 LLM 的 `temperature` 参数来控制回答的创造性
- 根据预测结果选择合适的向量数据库方案
- 集成到实际应用中

### 相关资源

- [XPULink 官网](https://www.xpulink.ai)
- [LlamaIndex 文档](https://docs.llamaindex.ai/)
- [BGE-M3 模型介绍](https://github.com/FlagOpen/FlagEmbedding)